
# Excel（.xlsm）VBA 一括エクスポート + 生成AI要約 + **業務別可視化** + **ワークシート関数分析**（プロキシ対応）

このノートブックは、次の処理をステップで実行します。

1. **高速エクスポート**：.xlsm から `.bas/.cls/.frm(+.frx)` を一括出力（UserForm以外は高速テキスト抽出）  
2. **生成AIの準備（Azure OpenAI）**：.env からプロキシ/キー/エンドポイント/バージョンを読み込み  
3. **モジュールごとの要約（VBA）**：役割 / 主要ロジック / 入出力 / イベント / 外部依存 を日本語で要約（※「改善点」「危険箇所」は含めません）  
4. **全体シーケンス図（Mermaid）**：モジュール関係の推定図を生成  
5. **ワークシート関数の収集・集計**：数式セル・名前定義・LAMBDA などを抽出・要約  
6. **業務別マッピング**：業務プロセスごとに、どのモジュール/シートがどの段でどんな役割を果たすかを整理（manual/auto/hybrid）  
7. **業務別フローチャート（Mermaid）**：業務×プログラム×シートの対応を flowchart で可視化  
8. **（任意）関数依存グラフ（Mermaid）**：シート間参照を graph で可視化  
9. **最終レポート（.md）生成**：上記を統合して1つのMarkdownに出力


In [7]:

# ==============================
# 設定
# ==============================
workbook_path = r"営業受注管理システム.xlsm"
output_dir = r".\vba_export_ai_v2"
final_report_name = "vba_project_summary.md"

include_sheet_functions = True
business_context_mode = "auto"           # "manual" | "auto" | "hybrid"
per_business_flowchart = True
function_dependency_graph = True
flowchart_orientation = "LR"
max_nodes_per_business = 15
show_confidence = False
business_context_yaml_path = r".\business_context.yaml"

import os, sys, traceback, time, re, csv, json, glob
from pathlib import Path
Path(output_dir).mkdir(parents=True, exist_ok=True)
print("Workbook Path:", os.path.abspath(workbook_path))
print("Output Dir   :", Path(output_dir).resolve())


Workbook Path: c:\Users\atsuk\OneDrive\ドキュメント\GitHub\Codex_VBA\営業受注管理システム.xlsm
Output Dir   : C:\Users\atsuk\OneDrive\ドキュメント\GitHub\Codex_VBA\vba_export_ai_v2


In [8]:

# 依存パッケージ
!pip install pywin32
!pip install python-dotenv
!pip install openai
!pip install pyyaml


In [9]:

try:
    import win32com.client as win32
except Exception:
    print("pywin32 の読み込みに失敗しました。'pip install pywin32' 実行後に再試行してください。")
    raise


In [10]:

# ========== 高速エクスポート ==========

CT_STD, CT_CLASS, CT_FORM, CT_DOC = 1, 2, 3, 100
EXT_MAP = {CT_STD: ".bas", CT_CLASS: ".cls", CT_FORM: ".frm", CT_DOC: ".cls"}

def safe_filename(name: str) -> str:
    return re.sub(r'[\\/:*?"<>|]', "_", name.strip())

def write_text(path, text, encoding="utf-8"):
    with open(path, "w", encoding=encoding, errors="ignore") as f:
        f.write(text)

def export_component_fast(comp, out_dir, use_export_for_forms=True):
    ctype = comp.Type
    name = comp.Name
    ext = EXT_MAP.get(ctype, ".cls")
    fname = safe_filename(f"{name}{ext}")
    fpath = os.path.join(out_dir, fname)

    stem, ext_only = os.path.splitext(fname)
    idx = 1
    while os.path.exists(fpath):
        fname = f"{stem}_{idx}{ext_only}"
        fpath = os.path.join(out_dir, fname)
        idx += 1

    if ctype == 3 and use_export_for_forms:
        comp.Export(os.path.abspath(fpath))
        return fpath
    else:
        cm = comp.CodeModule
        total_lines = cm.CountOfLines
        code = cm.Lines(1, total_lines) if total_lines > 0 else ""
        write_text(os.path.abspath(fpath), code)
        return fpath

def export_all_vba_fast(workbook_path: str, out_dir: str, verbose=True):
    t0 = time.time()
    excel = win32.DispatchEx("Excel.Application")
    excel.Visible = False
    try:
        excel.AutomationSecurity = 3
    except Exception:
        pass
    try:
        excel.DisplayAlerts = False
    except Exception:
        pass
    try:
        excel.AskToUpdateLinks = False
    except Exception:
        pass

    wb = None
    exported = []
    try:
        wb = excel.Workbooks.Open(os.path.abspath(workbook_path), UpdateLinks=0, ReadOnly=True)
        comps = wb.VBProject.VBComponents
        n = comps.Count
        if verbose:
            print(f"VBコンポーネント数: {n}")

        for i in range(1, n+1):
            comp = comps.Item(i)
            name = comp.Name
            ctype = comp.Type
            label = {1:"StdModule",2:"Class",3:"Form",100:"Document"}.get(ctype, str(ctype))

            t1 = time.time()
            out_path = export_component_fast(comp, out_dir, use_export_for_forms=True)
            dt = time.time() - t1

            print(f"[{i}/{n}] {name} ({label}) -> {out_path}  [{dt:.2f}s]")
            exported.append((name, ctype, out_path))

        manifest_csv = os.path.join(out_dir, "manifest.csv")
        with open(manifest_csv, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["ComponentName", "Type", "ExportPath"])
            w.writerows(exported)

        manifest_json = os.path.join(out_dir, "manifest.json")
        with open(manifest_json, "w", encoding="utf-8") as f:
            json.dump([{"name": n, "type": t, "path": p} for n, t, p in exported], f, ensure_ascii=False, indent=2)

        print(f"\nマニフェスト出力: {manifest_csv}, {manifest_json}")
        print(f"総所要時間: {time.time() - t0:.2f}s")
        return exported

    finally:
        if wb is not None:
            wb.Close(SaveChanges=False)
        excel.Quit()


In [11]:

# 実行
results = export_all_vba_fast(workbook_path, output_dir, verbose=True)

print("\nエクスポート結果サマリ:")
for name, ctype, path in results:
    print(f" - {name} (Type={ctype}) -> {path}")


VBコンポーネント数: 8
[1/8] ThisWorkbook (Document) -> .\vba_export_ai_v2\ThisWorkbook.cls  [0.02s]
[2/8] Sheet1 (Document) -> .\vba_export_ai_v2\Sheet1.cls  [0.01s]
[3/8] Sheet2 (Document) -> .\vba_export_ai_v2\Sheet2.cls  [0.01s]
[4/8] Sheet3 (Document) -> .\vba_export_ai_v2\Sheet3.cls  [0.01s]
[5/8] Sheet4 (Document) -> .\vba_export_ai_v2\Sheet4.cls  [0.01s]
[6/8] Sheet5 (Document) -> .\vba_export_ai_v2\Sheet5.cls  [0.01s]
[7/8] Sheet6 (Document) -> .\vba_export_ai_v2\Sheet6.cls  [0.01s]
[8/8] Sheet7 (Document) -> .\vba_export_ai_v2\Sheet7.cls  [0.01s]

マニフェスト出力: .\vba_export_ai_v2\manifest.csv, .\vba_export_ai_v2\manifest.json
総所要時間: 7.53s

エクスポート結果サマリ:
 - ThisWorkbook (Type=100) -> .\vba_export_ai_v2\ThisWorkbook.cls
 - Sheet1 (Type=100) -> .\vba_export_ai_v2\Sheet1.cls
 - Sheet2 (Type=100) -> .\vba_export_ai_v2\Sheet2.cls
 - Sheet3 (Type=100) -> .\vba_export_ai_v2\Sheet3.cls
 - Sheet4 (Type=100) -> .\vba_export_ai_v2\Sheet4.cls
 - Sheet5 (Type=100) -> .\vba_export_ai_v2\Sheet5.cls
 - She

In [12]:

# README を出力
readme_text = """【VBE への再インポート手順】
1) Excel（.xlsm）を開く -> Alt+F11 でVBEを開く
2) 対象プロジェクトを右クリック -> [ファイルのインポート]
3) .bas / .cls / .frm を選択してインポートする（フォームは .frx も必要）

【注意】
- ドキュメントモジュール（ThisWorkbook、Sheet1 など）は .cls として出力されます。
- 機密情報の外部送信に注意してください。
"""
readme_path = os.path.join(output_dir, "README.txt")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_text)
print("README を出力しました:", readme_path)


README を出力しました: .\vba_export_ai_v2\README.txt



## Azure OpenAI 準備（.env とプロキシ）

`.env` 例：

```
HTTP_PROXY=http://XXX.XX.X.XX:XXXX
HTTPS_PROXY=http://XXX.XX.XX.XX:XXXX
API_KEY=3pJrXXXXXXXXXXXXXXXX
API_VERSION=2025-01-01-preview
AZURE_ENDPOINT=https://XXXXXX.openai.azure.com/
```


In [13]:

from dotenv import load_dotenv
from openai import AzureOpenAI
import os

load_dotenv()

os.environ["http_proxy"]  = os.getenv("HTTP_PROXY", "")
os.environ["https_proxy"] = os.getenv("HTTPS_PROXY", "")

client = AzureOpenAI(
    api_key       = os.getenv("API_KEY"),
    api_version   = os.getenv("API_VERSION"),
    azure_endpoint= os.getenv("AZURE_ENDPOINT")
)

try:
    resp = client.chat.completions.create(
        model="gpt-4o-us-MEM-DX-openai-001",
        messages=[{"role": "user", "content": "接続テスト：1行で応答してください。"}],
        temperature=0
    )
    print(resp.choices[0].message.content)
except Exception as e:
    print("Azure OpenAI 接続テストに失敗:", e)


OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

In [ ]:

import re, json

def read_text_safely(path, max_chars=30000):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()[:max_chars]
    except Exception as e:
        return f"<<読み込みエラー: {e}>>"

def extract_procedures(code_text):
    proc_pat = re.compile(r'^\s*(Public|Private)?\s*(Sub|Function|Property\s+Get|Property\s+Let|Property\s+Set)\s+([A-Za-z0-9_]+)', re.IGNORECASE|re.MULTILINE)
    procs = []
    for m in proc_pat.finditer(code_text):
        name = m.group(3)
        kind = m.group(2)
        procs.append(f"{kind} {name}")
    return procs

def ensure_mermaid_fence(block: str, diagram_type_hint="sequenceDiagram"):
    if "```mermaid" not in block:
        return f"```mermaid\n{diagram_type_hint}\n{block}\n```"
    return block


## ステップ1：モジュール一覧の作成

In [ ]:

vba_files = []
for ext in ("*.bas", "*.cls", "*.frm"):
    vba_files.extend(glob.glob(os.path.join(output_dir, ext)))

modules_info = []
for fp in sorted(vba_files):
    name = os.path.splitext(os.path.basename(fp))[0]
    code_text = read_text_safely(fp, max_chars=60000)
    procs = extract_procedures(code_text)
    modules_info.append({
        "name": name,
        "path": fp,
        "procedures": procs[:200],
        "preview": code_text[:2000],
    })

print("モジュール数:", len(modules_info))
for m in modules_info[:5]:
    print("-", m["name"], f"(proc: {len(m['procedures'])})")


## ステップ2：各ファイルの要約（VBA）

In [ ]:

per_file_summaries = []

summary_dir = os.path.join(output_dir, "_summaries")
Path(summary_dir).mkdir(parents=True, exist_ok=True)

for m in modules_info:
    name = m["name"]
    path = m["path"]
    text = read_text_safely(path, max_chars=30000)

    system_prompt = "あなたはVBAコードのレビューアです。事実に基づき日本語で簡潔にまとめます。"
    user_prompt = f"""目的:
- このVBAモジュールの 役割/主要ロジック/入出力/イベント/外部依存 を日本語で簡潔に、箇条書き中心で要約してください。
- 不明な点は推測せず「不明」と書いてください。
- 出力は以下の形式で:
# {name} の要約
- 役割:
- 主要ロジック:
- 入出力:
- イベント:
- 外部依存:

--- VBAコード（先頭抜粋/最大30,000字） ---
{text}
"""

    try:
        res = client.chat.completions.create(
            model="gpt-4o-us-MEM-DX-openai-001",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_prompt},
            ],
            temperature=0
        )
        summary = res.choices[0].message.content
    except Exception as e:
        summary = f"<<要約エラー: {e}>>"

    per_file_summaries.append({"name": name, "path": path, "summary_text": summary})

    out_txt = os.path.join(summary_dir, f"{name}.summary.md")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(summary)
    print("要約出力:", out_txt)


## ステップ3：全体シーケンス図（VBA、Mermaid sequenceDiagram）

In [ ]:

module_briefs = [{"name": m["name"], "procedures": m["procedures"][:30]} for m in modules_info]

system_prompt = (
    "あなたはソフトウェアアーキテクトです。"
    "与えられた手がかりから合理的に流れを推定し、"
    "Mermaidのシーケンス図を日本語の注釈付きで作成します。"
    "不確実な関係は「(仮)」と注記してください。"
)
user_prompt = (
    "目的:\n"
    "- 下記のモジュール一覧と、その中の代表的なプロシージャ名から、想定される呼び出し関係・イベントの流れを推定し、Mermaidの sequenceDiagram を出力してください。\n"
    "- 不確実な関係は「(仮)」と注記してください。\n"
    "- できるだけ主要な処理の流れが一望できるよう、登場ノードは過不足なく整理してください。\n\n"
    "出力形式:\n"
    "```mermaid\n"
    "sequenceDiagram\n"
    "  %% ここに図を書く\n"
    "```\n\n"
    f"モジュール情報(要約):\n{json.dumps(module_briefs, ensure_ascii=False, indent=2)}"
)

try:
    res = client.chat.completions.create(
        model="gpt-4o-us-MEM-DX-openai-001",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt},
        ],
        temperature=0
    )
    mermaid_sequence_block = ensure_mermaid_fence(res.choices[0].message.content, "sequenceDiagram")
except Exception as e:
    mermaid_sequence_block = f"<<Mermaid生成エラー: {e}>>"

print(mermaid_sequence_block[:800], "...\n（全文は最終MDに挿入）")


## ステップ2c：ワークシート関数の抽出と集計

In [ ]:

sheet_funcs_summary = []
sheet_dependencies = set()

if include_sheet_functions:
    excel = win32.DispatchEx("Excel.Application")
    excel.Visible = False
    wb = None
    try:
        wb = excel.Workbooks.Open(os.path.abspath(workbook_path), UpdateLinks=0, ReadOnly=True)

        names_info = []
        try:
            for nm in wb.Names:
                names_info.append(("Workbook", nm.Name, getattr(nm, "RefersTo", "")))
        except Exception:
            pass
        for ws in wb.Worksheets:
            try:
                for nm in ws.Names:
                    names_info.append((ws.Name, nm.Name, getattr(nm, "RefersTo", "")))
            except Exception:
                pass

        for ws in wb.Worksheets:
            sh_name = ws.Name
            funcs_counter, templates_counter = {}, {}
            examples = []
            intra, inter, external = set(), set(), set()

            try:
                used = ws.UsedRange
                xlCellTypeFormulas = 16
                formula_cells = used.SpecialCells(xlCellTypeFormulas)
                areas = getattr(formula_cells, "Areas", [formula_cells])
                for area in areas:
                    for cell in area:
                        try:
                            f = str(getattr(cell, "Formula2", cell.Formula))
                        except Exception:
                            f = str(cell.Formula)
                        if not f or not f.startswith("="):
                            continue
                        if len(examples) < 5:
                            examples.append(f)

                        m = re.match(r"=([A-Z_][A-Z0-9_.]*)", f, flags=re.IGNORECASE)
                        if m:
                            fn = m.group(1).upper()
                            funcs_counter[fn] = funcs_counter.get(fn, 0) + 1

                        for m2 in re.finditer(r"\[([^\]]+)\]([^\]!]+)!", f):
                            external.add(m2.group(1))
                            dst = m2.group(2)
                            sheet_dependencies.add((sh_name, dst, "external"))
                        for m3 in re.finditer(r"(?:'([^']+)'|([A-Za-z0-9_]+))!", f):
                            dst = m3.group(1) or m3.group(2)
                            if dst and dst != sh_name:
                                inter.add(dst)
                                sheet_dependencies.add((sh_name, dst, "direct"))
                            else:
                                intra.add(sh_name)

                        if re.search(r"\bINDIRECT\b|\bOFFSET\b", f, flags=re.IGNORECASE):
                            sheet_dependencies.add((sh_name, "(不明:INDIRECT)", "indirect"))

                        tmpl = re.sub(r"\$?[A-Z]{1,3}\$?\d+", "CELL", f)
                        tmpl = re.sub(r"'[^']+'!", "SHEET!", tmpl)
                        tmpl = re.sub(r"[A-Za-z0-9_]+!", "SHEET!", tmpl)
                        templates_counter[tmpl] = templates_counter.get(tmpl, 0) + 1

            except Exception:
                pass

            def topn(d, n=10):
                return sorted(d.items(), key=lambda x: x[1], reverse=True)[:n]

            sheet_funcs_summary.append({
                "sheet": sh_name,
                "functions_topN": topn(funcs_counter, 10),
                "templates_topN": topn(templates_counter, 10),
                "refs": {
                    "intra": sorted(list(intra))[:10],
                    "inter": sorted(list(inter))[:20],
                    "external": sorted(list(external))[:10],
                },
                "role_hint": "",
                "examples": examples,
            })
    finally:
        if wb is not None:
            wb.Close(SaveChanges=False)
        excel.Quit()

print("シート数（関数集計）:", len(sheet_funcs_summary))


## ステップ2d：ワークシート関数の要約（シート別）

In [ ]:

sheet_summaries = []

if include_sheet_functions:
    for info in sheet_funcs_summary:
        sheet = info["sheet"]
        data_for_llm = {
            "sheet": sheet,
            "functions_topN": info["functions_topN"],
            "templates_topN": info["templates_topN"],
            "refs": info["refs"],
            "examples": info["examples"],
        }

        system_prompt = "あなたはExcelのワークシート設計を要約するアシスタントです。日本語で簡潔に説明します。"
        user_prompt = f"""目的:
- 次のシートが何の用途/役割を果たしていそうか、主要な関数/式テンプレの特徴、他シート/外部との入出力の観点で、短く日本語でまとめてください。
- 「改善点」「危険箇所」は書かないでください。
- 出力は以下の形式で:
# {sheet} シートの要約
- 用途/役割:
- 主要な関数・式テンプレの特徴:
- 他シート/外部との入出力:
データ（集計済み）:
{json.dumps(data_for_llm, ensure_ascii=False, indent=2)}
"""
        try:
            res = client.chat.completions.create(
                model="gpt-4o-us-MEM-DX-openai-001",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": user_prompt},
                ],
                temperature=0
            )
            summary = res.choices[0].message.content
        except Exception as e:
            summary = f"<<シート要約エラー: {e}>>"

        sheet_summaries.append({"sheet": sheet, "summary_text": summary})
        print(f"要約: {sheet}")


## ステップ3b：業務別マッピング（Business View）

In [ ]:

business_context = {"mode": business_context_mode, "processes": []}

def try_load_yaml(path):
    try:
        import yaml
        with open(path, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)
    except Exception as e:
        print("YAMLの読み込みに失敗:", e)
        return None

if business_context_mode == "manual":
    data = try_load_yaml(business_context_yaml_path)
    if data and "business_processes" in data:
        business_context["processes"] = data["business_processes"]
    else:
        print("manualモードですが、有効なYAMLが見つかりませんでした。空のまま続行します。")
else:
    compact_modules = [{"name": m["name"], "procedures": m["procedures"][:10]} for m in modules_info]
    compact_sheets = [{"sheet": s["sheet"], "functions_topN": s["functions_topN"][:5]} for s in sheet_funcs_summary]

    system_prompt = "あなたは製造・検査領域の業務整理を行うアナリストです。日本語で簡潔に出力します。"
    user_prompt = f"""目的:
- 次のモジュール一覧/シート関数サマリから、想定される主要な業務プロセスを3〜6件ほど推定し、YAMLで出力してください。
- それぞれについて name/goal/triggers/inputs/outputs/stakeholders/kpis を含めてください。
- 不確実なものは name に「(仮)」と付けてください。
- 改善点や危険箇所は書かないでください。

モジュール（要約）:
{json.dumps(compact_modules, ensure_ascii=False, indent=2)}

シート（関数サマリ抜粋）:
{json.dumps(compact_sheets, ensure_ascii=False, indent=2)}
"""
    try:
        res = client.chat.completions.create(
            model="gpt-4o-us-MEM-DX-openai-001",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_prompt},
            ],
            temperature=0
        )
        yaml_text = res.choices[0].message.content
    except Exception as e:
        yaml_text = f"<<業務候補生成エラー: {e}>>"

    if business_context_mode == "hybrid":
        try:
            with open(business_context_yaml_path, "w", encoding="utf-8") as f:
                f.write(yaml_text)
            print("業務候補（YAML）を出力しました。編集後に manual モードで再実行してください:", business_context_yaml_path)
        except Exception as e:
            print("YAML出力に失敗:", e)

    if business_context_mode == "auto":
        parsed = None
        try:
            import yaml
            parsed = yaml.safe_load(yaml_text)
        except Exception:
            parsed = None
        if parsed and "business_processes" in parsed:
            business_context["processes"] = parsed["business_processes"]
        else:
            print("autoモードでYAMLの解析に失敗しました。業務一覧は空のままで続行します。")

print("業務プロセス数:", len(business_context.get("processes", [])))


## ステップ3c：業務別フローチャート（Mermaid flowchart）

In [ ]:

per_business_flowcharts = {}

if per_business_flowchart and business_context.get("processes"):
    for proc in business_context["processes"][:10]:
        pname = proc.get("name", "業務(名称不明)")
        brief = {
            "process": {k: proc.get(k, "") for k in ["name", "goal", "triggers", "inputs", "outputs", "stakeholders", "kpis"]},
            "modules": [{"name": m["name"], "procedures": m["procedures"][:10]} for m in modules_info][:max_nodes_per_business],
            "sheets":  [{"sheet": s["sheet"], "functions_topN": s["functions_topN"][:5]} for s in sheet_funcs_summary][:max_nodes_per_business],
            "orientation": flowchart_orientation
        }
        system_prompt = (
            "あなたは業務プロセスとプログラム/シートの対応を設計するアーキテクトです。"
            "Mermaidのflowchartだけをコードブロックで出力します。"
            "不確実な箇所は(仮)と注記。日本語ラベルを使い、過剰なノードは避けてください。"
        )
        user_prompt = (
            "目的:\n"
            "- 次の業務プロセスに対して、関与するモジュール/シートを整理し、flowchart で示してください。\n"
            "- サブグラフ「業務: <name>」と「モジュール群」を用意し、ステップの流れと使用モジュール/シートの対応を矢印とラベルで表現してください。\n"
            f"- 図の向きは flowchart {flowchart_orientation} を使用してください。\n\n"
            "出力形式:\n"
            "```mermaid\n"
            f"flowchart {flowchart_orientation}\n"
            "  %% ここに図を書く\n"
            "```\n\n"
            f"データ:\n{json.dumps(brief, ensure_ascii=False, indent=2)}"
        )
        try:
            res = client.chat.completions.create(
                model="gpt-4o-us-MEM-DX-openai-001",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": user_prompt},
                ],
                temperature=0
            )
            block = ensure_mermaid_fence(res.choices[0].message.content, f"flowchart {flowchart_orientation}")
        except Exception as e:
            block = f"<<業務フローチャート生成エラー: {e}>>"
        per_business_flowcharts[pname] = block
        print(f"フローチャート生成: {pname} （先頭表示）\n", block[:400], "\n---")
else:
    print("業務プロセスが未定義のため、業務別フローチャートはスキップします。")


## ステップ3d：（任意）関数依存グラフ（Mermaid graph）

In [ ]:

function_graph_block = ""

if function_dependency_graph and include_sheet_functions:
    edges = []
    nodes = set()
    for src, dst, kind in sheet_dependencies:
        nodes.add(src); nodes.add(dst)
        label = "" if kind == "direct" else ("(仮:INDIRECT)" if kind == "indirect" else "(外部)")
        if label:
            edges.append(f'  "{src}" -->|{label}| "{dst}"')
        else:
            edges.append(f'  "{src}" --> "{dst}"')

    graph_lines = ["graph TD"]
    graph_lines.extend(sorted(set(edges)))
    function_graph_block = "```mermaid\n" + "\n".join(graph_lines) + "\n```"

    print(function_graph_block[:800], "...\n（全文は最終MDに挿入）")
else:
    print("関数依存グラフの生成はスキップします。")


## ステップ4：最終Markdown（レポート）生成

In [ ]:

def anchorify(name: str) -> str:
    return re.sub(r'[^a-z0-9\-]+', '', re.sub(r'\s+', '-', name.lower()))

toc_lines = []

header = [
    "# VBA プロジェクト総合レポート",
    "",
    "## 目次",
]

main_sections = [
    "シーケンス図（Mermaid推定：全体）",
    "業務別マッピング（Business View）",
    "ワークシート関数の分析（Sheet Functions Overview）" if include_sheet_functions else None,
    "各モジュールの要約",
]
main_sections = [s for s in main_sections if s]
for s in main_sections:
    header.append(f"- [{s}](#{anchorify(s)})")

body = []
body.append(f"\n## {main_sections[0]}")
body.append(mermaid_sequence_block)

body.append(f"\n## {main_sections[1]}")
if business_context.get("processes"):
    body.append("### 業務一覧")
    for p in business_context["processes"]:
        nm = p.get("name", "名称不明"); goal = p.get("goal", "")
        body.append(f"- **{nm}**  — 目的: {goal}")
else:
    body.append("_業務一覧は未定義です。_")

if per_business_flowchart and business_context.get("processes"):
    body.append("\n### 業務別フローチャート（Mermaid）")
    for p in business_context["processes"]:
        pname = p.get("name", "業務(名称不明)")
        block = per_business_flowcharts.get(pname, "")
        if block:
            body.append(f"\n#### {pname}\n{block}")
        else:
            body.append(f"\n#### {pname}\n_図は生成されませんでした。_")

if include_sheet_functions:
    body.append(f"\n## ワークシート関数の分析（Sheet Functions Overview）")
    if function_dependency_graph and function_graph_block:
        body.append("\n### 関数依存グラフ")
        body.append(function_graph_block)
    if sheet_summaries:
        body.append("\n### シート別要約")
        for s in sheet_summaries:
            body.append(s["summary_text"]); body.append("\n---")
    else:
        body.append("_シート要約はありません。_")

body.append(f"\n## 各モジュールの要約")
for item in per_file_summaries:
    body.append(item["summary_text"]); body.append("\n---\n")

final_md = "\n".join(header + [""] + body)

final_md_path = os.path.join(output_dir, final_report_name)
with open(final_md_path, "w", encoding="utf-8") as f:
    f.write(final_md)

print("最終Markdownを出力しました:", final_md_path)


> **補足**  
> Mermaid図は推定です。必要に応じて調整してください。『改善点』『危険箇所』は出力しません。